## This notebook was for the first cut solution of a CNN + for evaluating the different dataset we generated from our data in accordance with the CNN predictions

In [20]:
import pandas as pd
from tensorflow.keras.layers import Dense, Embedding,GlobalMaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
import numpy as np
import pandas as pd
from gensim.models.word2vec import Word2Vec
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Conv1D, MaxPool1D, GlobalMaxPool1D, Embedding, Activation
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer
from sklearn import preprocessing
import pickle
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

# Preprocessing and model is used from 
# https://www.kaggle.com/code/jagannathrk/word2vec-cnn-text-classification

## Adapt the preprocessing pipeline to the songs so thath the input is cleaned

In [21]:
import spacy

def tokenization(text):
    """Use this function to tokenize text.

    :param text: Text as list
    :type text: list[spacy.tokens.token.Token]
    :return: Tokenized text as list
    :rtype: list[spacy.tokens.token.Token]
    """

    token_list = []
    for doc in text: 
        # iterate over tokens in docs
        for token in doc:
            token_list.append(token)

    return token_list


def stop_word_removal(text): 
    """Use this function to remove stop words. 

    :param text: Tokens to remove stop words from 
    :type text: list[spacy.tokens.token.Token]
    :return: Tokens without stop words
    :rtype: list[spacy.tokens.token.Token]
    """

    token_list_without_stop = []
    # Don't add token to list if stop word
    for token in text:
        if token.is_stop == False: 
            token_list_without_stop.append(token)

    return token_list_without_stop


def punctutation_removal(text): 
    """Use this function to remove punctuation.

    :param text: Tokens to remove punctuation from
    :type text: list[spacy.tokens.token.Token]
    :return: Tokens without punctuation
    :rtype: list[spacy.tokens.token.Token]
    """

    token_list_no_stop_no_punct = []
    # Don't add token to list if punctuation
    for token in text:
        if token.is_punct == False:
            token_list_no_stop_no_punct.append(token)

    return token_list_no_stop_no_punct


def lemmatization(text): 
    """Use this function to lemmatize a given text.

    :param text: Tokens to lemmatize
    :type text: list[spacy.tokens.token.Token]
    :return: lemmatized tokens
    :rtype: list[str]
    """

    token_list_no_stop_no_punct_lemmatized = []
    for token in text: 
        if "\n" not in token.lemma_:
            token_list_no_stop_no_punct_lemmatized.append(token.lemma_)
    return token_list_no_stop_no_punct_lemmatized


def processing_pipeline(song_data):
    """Use this function to execute the entire processing pipeline on given song data.
    Preprocessing steps:
    - Tokenization
    - Stop word removal
    - Punctuation removal
    - Lemmatization
    - ...

    :param song_data: song data saved in a json file containing song name, artist name and lyrics
    :type song_data: dict
    :return: preprocessed song data
    :rtype: dict
    """

    nlp = spacy.load("en_core_web_sm", disable = ['ner'])
    
    for row in range(len(song_data)):
        text_nlp_pipe = list(nlp.pipe([song_data.iloc[row]["Lyric"]]))
    
        # Tokenization
        song_data.at[row,"Lyric"] = tokenization(text_nlp_pipe)
        # Stop word removal
        song_data.at[row,"Lyric"] = stop_word_removal(song_data.iloc[row]["Lyric"])
        # Punctuation removal
        song_data.at[row,"Lyric"] = punctutation_removal(song_data.iloc[row]["Lyric"])
        # Lemmatization
        song_data.at[row,"Lyric"] = lemmatization(song_data.iloc[row]["Lyric"])
        song_data.at[row,"Lyric"] = " ".join(song_data.iloc[row]["Lyric"])
        song_data.at[row,"Lyric"] = song_data.iloc[row]["Lyric"].lower()
        if row%500 == 0 and row >= 500:
            print(f"processsed: {row} rows out of {len(song_data)}")
    return song_data


In [22]:
# helper function to use the word2vec as an layer in keras 
# taken from the gensim wikipage: https://github.com/RaRe-Technologies/gensim/wiki/Using-Gensim-Embeddings-with-Keras-and-Tensorflow

from tensorflow.keras.layers import Embedding

def gensim_to_keras_embedding(model, train_embeddings=False):
    """Get a Keras 'Embedding' layer with weights set from Word2Vec model's learned word embeddings.

    Parameters
    ----------
    train_embeddings : bool
        If False, the returned weights are frozen and stopped from being updated.
        If True, the weights can / will be further updated in Keras.

    Returns
    -------
    `keras.layers.Embedding`
        Embedding layer, to be used as input to deeper network layers.

    """
    keyed_vectors = model.wv  # structure holding the result of training
    weights = keyed_vectors.vectors  # vectors themselves, a 2D numpy array    
    index_to_key = keyed_vectors.index_to_key  # which row in `weights` corresponds to which word?

    layer = Embedding(
        input_dim=weights.shape[0],
        output_dim=weights.shape[1],
        weights=[weights],
        trainable=train_embeddings,
    )
    return layer

In [36]:
def create_model(mood_count,filters,multiplicator, multiplicator2, kernel, adapt_embedding, outputfunction):
    
    keras_model = Sequential()
    keras_model.add(gensim_to_keras_embedding(word2vec_model, True))
    keras_model.add(Dropout(0.2))
    keras_model.add(Conv1D(filters, kernel, activation='relu', padding='same', strides=1))
    keras_model.add(Conv1D(filters, kernel, activation='relu', padding='same', strides=1))
    keras_model.add(MaxPool1D())
    keras_model.add(Dropout(0.2))
    keras_model.add(Conv1D(filters*multiplicator, kernel, activation='relu', padding='same', strides=1))
    keras_model.add(Conv1D(filters*multiplicator, kernel, activation='relu', padding='same', strides=1))
    keras_model.add(MaxPool1D())
    keras_model.add(Dropout(0.2))
    keras_model.add(Conv1D(filters*multiplicator2, kernel, activation='relu', padding='same', strides=1))
    keras_model.add(Conv1D(filters*multiplicator2, kernel, activation='relu', padding='same', strides=1))
    keras_model.add(GlobalMaxPool1D())
    keras_model.add(Dropout(0.2))
    keras_model.add(Dense(200))
    keras_model.add(Activation('relu'))
    keras_model.add(Dropout(0.2))
    # Number of moods to be classified to
    keras_model.add(Dense(mood_count))
    keras_model.add(Activation(outputfunction))
    
    keras_model.compile(loss='binary_crossentropy', metrics=['acc'], optimizer='adam')
    return keras_model


In [46]:
def test_dataset(dataset_path, mood_count):
    print(f"Testing now dataset {dataset_path} with {mood_count} moods")
    df = pd.read_csv(dataset_path)
    print((df["Mood"].unique()))
    df = processing_pipeline(df)
    list(df["Lyric"].head(2))
    # merge lyrics together
    lyrics = []
    for i in df['Lyric']:
        lyrics.append(i.split())
    print(lyrics[:2])
    
    # train the word2vec model
    # vector size according to https://moj-analytical-services.github.io/NLP-guidance/NNmodels.html#:~:text=The%20standard%20Word2Vec%20pre%2Dtrained,fewer%20dimensions%20to%20represent%20them.
    # mincount = 2 to prevent misspellings
    word2vec_model = Word2Vec(lyrics, vector_size=150, window=5, min_count=2, workers=16)
    
    # use the keras tokenizer and apply it to the lyrics
    # number in first row is vocab size from word2vec model
    token = Tokenizer(len(word2vec_model.wv))
    token.fit_on_texts(df['Lyric'])
    text = token.texts_to_sequences(df['Lyric'])
    text = pad_sequences(text, 180)
    print(text[3:5])
    
    # encode the labels
    le = preprocessing.LabelEncoder()
    y = le.fit_transform(df['Mood'])
    y = to_categorical(y)
    # save the label encoder
    np.save('label_encoder.npy', le.classes_)
    
    x_train, x_test, y_train, y_test = train_test_split(np.array(text), y, test_size=0.2, stratify=y)
    
    seed = 7
    np.random.seed(seed)
    batch_size = 32
    epochs = 3

    model_CV = KerasClassifier(build_fn=create_model, epochs=epochs, 
                               batch_size=batch_size, verbose=1)
    # grid search parameters
    filters = [50, 100]
    multiplicator = [2]
    multiplicator2 = [3,4]
    kernel = [5]
    adapt_embedding = [True, False]
    outputfunction = ["relu","softmax"]

    param_grid = dict(mood_count=[mood_count], filters=filters, multiplicator=multiplicator, multiplicator2=multiplicator2, kernel=kernel, adapt_embedding=adapt_embedding, outputfunction=outputfunction)
    grid = GridSearchCV(estimator=model_CV, param_grid=param_grid, n_jobs=-1, cv=2)
    grid_result = grid.fit(x_train, y_train)
    
    
    # print results
    print(f'Best Accuracy for dataset {dataset_path} is: {grid_result.best_score_} using {grid_result.best_params_}')
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print(f' mean={mean:.4}, std={stdev:.4} using {param}')
    print("##"*50)


In [47]:
input_data = [
    ["./data/song-data-labels-cleaned.csv",17],
    ["./data/song-data-labels-cleaned-quadrant.csv",4],
    ["./data/song-data-labels-cleaned-seven-moods.csv",7]
]
for i in input_data:
    test_dataset(i[0], i[1])

Testing now dataset ./data/song-data-labels-cleaned.csv with 17 moods
['romantic' 'happy' 'sad' 'grief' 'upbeat' 'earnest' 'cheerful'
 'depressed' 'anger' 'calm' 'excitement' 'aggression' 'confident' 'angst'
 'brooding' 'desire' 'pessimism']
processsed: 500 rows out of 27705
processsed: 1000 rows out of 27705
processsed: 1500 rows out of 27705
processsed: 2000 rows out of 27705
processsed: 2500 rows out of 27705
processsed: 3000 rows out of 27705
processsed: 3500 rows out of 27705
processsed: 4000 rows out of 27705
processsed: 4500 rows out of 27705
processsed: 5000 rows out of 27705
processsed: 5500 rows out of 27705
processsed: 6000 rows out of 27705
processsed: 6500 rows out of 27705
processsed: 7000 rows out of 27705
processsed: 7500 rows out of 27705
processsed: 8000 rows out of 27705
processsed: 8500 rows out of 27705
processsed: 9000 rows out of 27705
processsed: 9500 rows out of 27705
processsed: 10000 rows out of 27705
processsed: 10500 rows out of 27705
processsed: 11000 rows

/tmp/ipykernel_1834/658696831.py:44: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_CV = KerasClassifier(build_fn=create_model, epochs=epochs,
2023-02-18 09:47:19.426113: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-18 09:47:19.455471: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-18 09:47:19

2023-02-18 09:47:21.183100: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-18 09:47:21.183772: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-18 09:47:21.183863: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-02-18 09:47:21.191589: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory


2023-02-18 09:47:23.205979: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-18 09:47:23.206022: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-18 09:47:23.206057: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-D6IC8GJA): /proc/driver/nvidia/version does not exist
2023-02-18 09:47:23.206326: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/3
Epoch 1/3
Epoch 1/3
Epoch 1/3
Epoch 1/3
Epoch 1/3
Epoch 1/3
Epoch 1/3
347/347 [==============================] - 247s 700ms/step - loss: 0.4585 - acc: 0.2495
Epoch 2/3
347/347 [==============================] - 248s 705ms/step - loss: 0.1941 - acc: 0.2601
Epoch 2/3
347/347 [==============================] - 265s 753ms/step - loss: 0.1883 - acc: 0.2632
Epoch 2/3
347/347 [==============================] - 272s 772ms/step - loss: 0.2887 - acc: 0.2716
Epoch 2/3
347/347 [==============================] - 243s 701ms/step - loss: 0.1788 - acc: 0.2920
Epoch 3/3
347/347 [==============================] - 243s 700ms/step - loss: 0.2208 - acc: 0.2949
Epoch 3/3
347/347 [==============================] - 243s 701ms/step - loss: 0.1796 - acc: 0.2803
Epoch 3/3
347/347 [==============================] - 268s 772ms/step - loss: 0.1784 - acc: 0.2936
Epoch 3/3
347/347 [==============================] - 266s 766ms/step - loss: 0.3147 - acc: 0.2894
Epoch 3/3
347/347 [=============================

...

Testing now dataset ./data/song-data-labels-cleaned-seven-moods.csv with 7 moods
['romantic' 'happy' 'grief' 'enthusiastic' 'calm' 'depressed' 'anger']
processsed: 500 rows out of 19576
processsed: 1000 rows out of 19576
processsed: 1500 rows out of 19576
processsed: 2000 rows out of 19576
processsed: 2500 rows out of 19576
processsed: 3000 rows out of 19576
processsed: 3500 rows out of 19576
processsed: 4000 rows out of 19576
processsed: 4500 rows out of 19576
processsed: 5000 rows out of 19576
processsed: 5500 rows out of 19576
processsed: 6000 rows out of 19576
processsed: 6500 rows out of 19576
processsed: 7000 rows out of 19576
processsed: 7500 rows out of 19576
processsed: 8000 rows out of 19576
processsed: 8500 rows out of 19576
processsed: 9000 rows out of 19576
processsed: 9500 rows out of 19576
processsed: 10000 rows out of 19576
processsed: 10500 rows out of 19576
processsed: 11000 rows out of 19576
processsed: 11500 rows out of 19576
processsed: 12000 rows out of 19576
proc

/tmp/ipykernel_1834/658696831.py:44: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_CV = KerasClassifier(build_fn=create_model, epochs=epochs,
2023-02-18 12:00:18.922070: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-18 12:00:18.970399: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-18 12:00:19

2023-02-18 12:00:20.617520: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-18 12:00:20.618152: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-18 12:00:20.618231: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-02-18 12:00:20.647516: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory


Epoch 1/3
Epoch 1/3
Epoch 1/3
Epoch 1/3
Epoch 1/3
Epoch 1/3
Epoch 1/3
Epoch 1/3
245/245 [==============================] - 170s 681ms/step - loss: 0.7688 - acc: 0.1530
Epoch 2/3
245/245 [==============================] - 173s 692ms/step - loss: 0.7028 - acc: 0.1493
Epoch 2/3
Epoch 2/3
245/245 [==============================] - 173s 695ms/step - loss: 0.4204 - acc: 0.1527
Epoch 2/3
245/245 [==============================] - 188s 748ms/step - loss: 0.9947 - acc: 0.1584
Epoch 2/3
245/245 [==============================] - 190s 760ms/step - loss: 0.8498 - acc: 0.1481
Epoch 2/3
245/245 [==============================] - 192s 767ms/step - loss: 0.4184 - acc: 0.1540
Epoch 2/3
245/245 [==============================] - 161s 658ms/step - loss: 0.4101 - acc: 0.1602
Epoch 3/3
245/245 [==============================] - 164s 667ms/step - loss: 0.4094 - acc: 0.1794
Epoch 3/3
245/245 [==============================] - 163s 666ms/step - loss: 0.4077 - acc: 0.1891
Epoch 3/3
245/245 [===================

245/245 [==============================] - 108s 440ms/step - loss: 0.4085 - acc: 0.1889
Epoch 3/3
245/245 [==============================] - 113s 460ms/step - loss: 0.4118 - acc: 0.1642
Epoch 3/3
245/245 [==============================] - 171s 596ms/step - loss: 0.4177 - acc: 0.1517
Epoch 2/3
245/245 [==============================] - 172s 598ms/step - loss: 0.4188 - acc: 0.1469
Epoch 2/3
245/245 [==============================] - 174s 607ms/step - loss: 1.1632 - acc: 0.1616
Epoch 2/3
245/245 [==============================] - 176s 611ms/step - loss: 0.7862 - acc: 0.1582
Epoch 2/3
245/245 [==============================] - 146s 595ms/step - loss: 0.4126 - acc: 0.1552
Epoch 3/3
245/245 [==============================] - 147s 601ms/step - loss: 0.4112 - acc: 0.1635
Epoch 3/3
245/245 [==============================] - 147s 599ms/step - loss: 0.9381 - acc: 0.1631
Epoch 3/3
245/245 [==============================] - 147s 601ms/step - loss: 0.5900 - acc: 0.1564
Epoch 3/3
245/245 [===========

# Sample prediction
Code might be broken due to heavy restructuring of the notebook, but it should only assist as a "documentation" for us developers anyway

In [84]:
import spacy

def tokenization(text: list[spacy.tokens.token.Token]) -> list[spacy.tokens.token.Token]:
    """Use this function to tokenize text.

    :param text: Text as list
    :type text: list[spacy.tokens.token.Token]
    :return: Tokenized text as list
    :rtype: list[spacy.tokens.token.Token]
    """

    token_list = []
    for doc in text: 
        # iterate over tokens in docs
        for token in doc:
            token_list.append(token)

    return token_list


def stop_word_removal(text: list[spacy.tokens.token.Token]) -> list[spacy.tokens.token.Token]: 
    """Use this function to remove stop words. 

    :param text: Tokens to remove stop words from 
    :type text: list[spacy.tokens.token.Token]
    :return: Tokens without stop words
    :rtype: list[spacy.tokens.token.Token]
    """

    token_list_without_stop = []
    # Don't add token to list if stop word
    for token in text:
        if token.is_stop == False: 
            token_list_without_stop.append(token)

    return token_list_without_stop


def punctutation_removal(text: list[spacy.tokens.token.Token]) -> list[spacy.tokens.token.Token]: 
    """Use this function to remove punctuation.

    :param text: Tokens to remove punctuation from
    :type text: list[spacy.tokens.token.Token]
    :return: Tokens without punctuation
    :rtype: list[spacy.tokens.token.Token]
    """

    token_list_no_stop_no_punct = []
    # Don't add token to list if punctuation
    for token in text:
        if token.is_punct == False:
            token_list_no_stop_no_punct.append(token)

    return token_list_no_stop_no_punct


def lemmatization(text: list[spacy.tokens.token.Token]) -> list[str]: 
    """Use this function to lemmatize a given text.

    :param text: Tokens to lemmatize
    :type text: list[spacy.tokens.token.Token]
    :return: lemmatized tokens
    :rtype: list[str]
    """

    token_list_no_stop_no_punct_lemmatized = []
    for token in text: 
        if "\n" not in token.lemma_:
            token_list_no_stop_no_punct_lemmatized.append(token.lemma_)
    return token_list_no_stop_no_punct_lemmatized


def processing_pipeline(song_data: dict) -> dict:
    """Use this function to execute the entire processing pipeline on given song data.
    Preprocessing steps:
    - Tokenization
    - Stop word removal
    - Punctuation removal
    - Lemmatization
    - ...

    :param song_data: song data saved in a json file containing song name, artist name and lyrics
    :type song_data: dict
    :return: preprocessed song data
    :rtype: dict
    """

    nlp = spacy.load("en_core_web_sm", disable = ['ner'])
    text_nlp_pipe = list(nlp.pipe([song_data["Lyrics"]]))
    
    # Tokenization
    song_data["Lyrics"] = tokenization(text_nlp_pipe)
    # Stop word removal
    song_data["Lyrics"] = stop_word_removal(song_data["Lyrics"])
    # Punctuation removal
    song_data["Lyrics"] = punctutation_removal(song_data["Lyrics"])
    # Lemmatization
    song_data["Lyrics"] = lemmatization(song_data["Lyrics"])

    return song_data


In [122]:
# test model:
test_song = {
        "Song": "Mockingbird",
        "Artist": "Eminem",
        "Lyrics": (
            """Yeah
            I know sometimes things may not always make sense to you right now
            But hey, what daddy always tell you?
            Straighten up little soldier
            Stiffen up that upper lip
            What you crying about?
            You got me
            Hailie, I know you miss your mom, and I know you miss your dad
            When I'm gone, but I'm trying to give you the life that I never had
            I can see you're sad, even when you smile, even when you laugh
            I can see it in your eyes, deep inside you want to cry
            'Cause you're scared, I ain't there, daddy's with you in your prayers
            No more crying, wipe them tears, daddy's here, no more nightmares
            We gon' pull together through it, we gon' do it
            Laney uncle's crazy, ain't he? Yeah, but he loves you girl and you better know it
            We're all we got in this world, when it spins, when it swirls
            When it whirls, when it twirls, two little beautiful girls
            Lookin' puzzled, in a daze, I know it's confusing you
            Daddy's always on the move, mamma's always on the news
            I try to keep you sheltered from it, but somehow it seems
            The harder that I try to do that, the more it backfires on me
            All the things growing up, his daddy, daddy had to see
            Daddy don't want you to see, but you see just as much as he did
            We did not plan it to be this way, your mother and me
            But things have gotten so bad between us, I don't see us ever being together
            Ever again like we used to be when we was teenagers
            But then of course everything always happens for a reason
            I guess it was never meant to be
            But it's just something we have no control, over and that's what destiny is
            But no more worries, rest your head and go to sleep
            Maybe one day we'll wake up, and this will all just be a dream
            Now hush little baby, don't you cry
            Everything's gonna be alright
            Stiffen that upper-lip up, little lady, I told ya
            Daddy's here to hold ya through the night
            I know mommy's not here right now, and we don't know why
            We fear how we feel inside
            It may seem a little crazy, pretty baby
            But I promise momma's gon' be alright
            Huh, it's funny
            I remember back one year when daddy had no money
            Mommy wrapped the Christmas presents up and stuck 'em under the tree
            And said, "Some of 'em were from me, 'cause Daddy couldn't buy 'em"
            I'll never forget that Christmas, I sat up the whole night crying
            'Cause daddy felt like a bum
            See daddy had a job
            But his job was to keep the food on the table for you and mom
            And at the time every house that we lived in
            Either kept getting broke into and robbed
            Or shot up on the block
            And your Mom was saving money for you in a jar
            Tryna start a piggy bank for you, so you could go to college
            Almost had a thousand dollars 'til someone broke in and stole it
            And I know it hurt so bad, it broke your momma's heart
            And it seemed like everything was just startin' to fall apart
            Mom and dad was arguin' a lot, so momma moved back
            On the Chalmers in the flat one-bedroom apartment
            And dad moved back to the other side of 8 Mile on Novara
            And that's when daddy went to California with his C.D
            And met Dr. Dre, and flew you and momma out to see me
            But daddy had to work, you and momma had to leave me
            Then you started seeing daddy on the T.V
            And momma didn't like it, and you and Laney were to young to understand it
            Papa was a rollin' stone, momma developed a habit
            And it all happened too fast for either one of us to grab it
            I'm just sorry you were there and had to witness it first hand
            'Cause all I ever wanted to do was just make you proud
            Now I'm sittin' in this empty house
            Just reminiscing, lookin' at your baby pictures
            It just trips me out
            To see how much you both have grown
            It's almost like you're sisters now
            Wow, guess you pretty much are, and daddy's still here
            Laney, I'm talkin' to you too, daddy's still here
            I like the sound of that, yeah, It's got a ring to it, don't it?
            Shh, momma's only gone for the moment
            Now hush little baby, don't you cry
            Everything's gonna be alright
            Stiffen that upper-lip up, little lady, I told ya
            Daddy's here to hold ya through the night
            I know mommy's not here right now, and we don't know why
            We fear how we feel inside
            It may seem a little crazy, pretty baby
            But I promise, momma's gon' be alright
            And if you ask me too
            Daddy's gonna buy you a Mockingbird
            I'ma give you the world
            I'ma buy a diamond ring for you, I'ma sing for you
            I'll do anything for you to see you smile
            And if that Mockingbird don't sing, and that ring don't shine
            I'ma break that birdies neck
            I'd go back to the jeweler who sold it to ya
            And make him eat every karat, don't fuck with dad (haha)"""
        ),
}


In [123]:
processed_song = processing_pipeline(test_song)

In [124]:
print(processed_song["Lyrics"])

['yeah', 'know', 'thing', 'sense', 'right', 'hey', 'daddy', 'tell', 'straighten', 'little', 'soldier', 'Stiffen', 'upper', 'lip', 'cry', 'get', 'Hailie', 'know', 'miss', 'mom', 'know', 'miss', 'dad', 'go', 'try', 'life', 'sad', 'smile', 'laugh', 'eye', 'deep', 'inside', 'want', 'cry', "'cause", 'scared', 'be', 'daddy', 'prayer', 'crying', 'wipe', 'tear', 'daddy', 'nightmare', 'gon', 'pull', 'gon', 'Laney', 'uncle', 'crazy', 'be', 'yeah', 'love', 'girl', 'well', 'know', 'get', 'world', 'spin', 'swirl', 'whirl', 'twirl', 'little', 'beautiful', 'girl', 'Lookin', 'puzzle', 'daze', 'know', 'confuse', 'Daddy', 'mamma', 'news', 'try', 'shelter', 'hard', 'try', 'backfire', 'thing', 'grow', 'daddy', 'daddy', 'daddy', 'want', 'plan', 'way', 'mother', 'thing', 'get', 'bad', 'like', 'teenager', 'course', 'happen', 'reason', 'guess', 'mean', 'control', 'destiny', 'worry', 'rest', 'head', 'sleep', 'maybe', 'day', 'wake', 'dream', 'hush', 'little', 'baby', 'cry', 'go', 'to', 'alright', 'Stiffen', 'up

In [176]:
#load the tokenizer
with open('tokenizer.pickle', 'rb') as handle:
    token = pickle.load(handle)
# tokenize
text = token.texts_to_sequences([processed_song["Lyrics"]])
text = pad_sequences(text, 180)

In [177]:
print(text)

[[  179    26    60    62   447    10     3  1320   447   775   775  1071
    855  1017     8   265    41   218  1169    53  1722   413   869  1017
    156   211  2485  1636    82  6451  1412  3248   459   900   285    53
    371     2   136    98    53  1139    21     3  2863    37   258  1017
    906 17626   304  1139   255  1561  1258  2518   906   255  1689   392
    447     4   929   147  2364   126  1139   447   150  1139    24    82
     73   447  1139     3 19532   189   162  1033  1047   314  1139  6769
   1766   232   242   702   291  1315    49    62     9   757  1193   265
  14576   528    14   365   664   180     3   601  1212   208   267   447
  19532   603   447     3   167    18     6   317  6076  1139     4   249
   1099    43    14    60     4    13   166  9315  3856   334    43   313
     20   110   447    40   110    26     2  2008    27     2   159    10
     64    43   182   267    14   262  1139   571   166   140   447     4
     13   349 12588    33   349   638 

In [178]:
pred = keras_model.predict(text)

1/1 [==============================] - 0s 21ms/step


In [179]:
print(pred)
# propability distr.

[[0.03035141 0.07305858 0.01072862 0.00733498 0.10687384 0.01872347
  0.00504669 0.09882194 0.00760916 0.00933376 0.00872805 0.02308791
  0.19617064 0.00064153 0.08658496 0.19915532 0.11774925]]


In [180]:
pred_mood=np.argmax(pred,axis=1)
# get the mood that is predicted the most 

In [181]:
print(pred_mood)

[15]


In [171]:
# load the label encoder
encoder = preprocessing.LabelEncoder()
encoder.classes_ = numpy.load('label_encoder.npy', allow_pickle=True)

print(encoder.inverse_transform(pred_mood))
# reverse transform the mood

['sad']
